In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)

In [ ]:
engine

In [ ]:
engine.execute('SELECT * FROM station LIMIT 5').fetchall()

In [ ]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [ ]:
session

In [ ]:
# Get a list of column names and types
#columns = inspector.get_columns('na')
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Get a list of column names and types
#columns = inspector.get_columns('na')
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
## cross multiplication of 2 tables and filtering
## peprformance overhead sql optimize rdbms obdms sql alchemy

same_station = session.query(Measurement,Station).filter(Measurement.prcp == Station.station).limit(10).all()


for record in same_station:
    (measurement, station) = record
   
    print(measurement.station, Station.station)

In [ ]:
# Earliest Date
session.query(Measurement.date).order_by(Measurement.date).first()

In [ ]:
# Latest Date
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
session.query(Measurement.date).\
    filter(Measurement.date > '2011-03-01').\
    order_by(Measurement.date).all()

# Exploratory Climate Analysis

#### Basic Calculations for plotting
#### Calculationg the precipitaiton for the last 12 months 
#### Finding the last data point for getting 1 year

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
month_presp= session.query(Measurement.date).order_by(Measurement.id.desc()).first()
print(month_presp)


# Calculate the date 1 year ago from the last data point in the database
last_year = dt.date(2017,8,23) - dt.timedelta(days=365)
print(last_year)

# Perform a query to retrieve the data and precipitation scores
rain = session.query(Measurement.date, Measurement.prcp).\
filter(Measurement.date >= last_year).\
order_by(Measurement.date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
presp_df = pd.DataFrame(rain)
presp_df.head()
# Sort the dataframe by date
presp_df.set_index("date").head()

# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
presp_df = pd.DataFrame(rain)
presp_df.head()
# Sort the dataframe by date
presp_df.set_index("date").head()


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
presp_df.plot("date", "prcp")
plt.xlabel("Date")
plt.legend(["Precipitation"])
plt.rcParams["figure.figsize"] = (15,8)
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


In [ ]:
presp_df.describe()

![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?
num_locations = session.query(Measurement).group_by(Measurement.station).count()
num_locations

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
list_st= session.query(Measurement.tobs).order_by(Measurement.tobs.desc()).first()
print(list_st)

In [ ]:

active_station = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).\
   order_by(func.count(Measurement.tobs).desc()).all()
active_station

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
best_station = active_station[0][0]
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
   filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogr
hitemp_obsrv= session.query(Measurement.station, Measurement.tobs).\
filter(Measurement.station  == 'USC00519281').\
filter(Measurement.date > last_year).all()

In [ ]:
hitemp_obsrv


In [ ]:
tobs_df = pd.DataFrame(hitemp_obsrv)
tobs_df.set_index('station', inplace=True)
tobs_df.head()

In [ ]:
plt.hist(tobs_df['tobs'],12)
plt.xlabel('Temp')
#plt.hist(tobs_df, bins=20)
plt.ylabel('Frequency')
plt.legend(['tobs'])
plt.tight_layout()

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 

def daily_normals(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date == start_date).filter(Measurement.date == end_date).all()
# function usage example
print(daily_normals('01','01'))
# for your trip using the previous year's data for those same dates.


In [ ]:
def daily_normals(date):
    """Daily Normals. 
    Args: date (str): A date string in the format '%m-%d' 
    Returns: A list of tuples containing the daily normals, tmin, tavg, and tmax
    """

    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
# function usage example
print(daily_normals("01-01"))

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
